In [73]:
import sys,os,re
import numpy as np
from pdastro import pdastroclass, pdastrostatsclass,unique,rmfile
import matplotlib.pyplot as plt
from   matplotlib.ticker import FormatStrFormatter,MultipleLocator

import pysiaf

v2v3 =  pdastrostatsclass()
v2v3.load('nircam_v2v3_v3.txt')
for ix_nrc in v2v3.getindices():
    #print(f'\n############################################ \n',v2v3.t.loc[ix_nrc,"nrc_filename"])
    m = re.search('^jw(\d\d\d\d\d)',os.path.basename(v2v3.t.loc[ix_nrc,"nrc_filename"]))
    if m is not None:
        progID = m.groups()[0]
    else:
        progID = None
    v2v3.t.loc[ix_nrc,'progID']=progID
#v2v3.write()

nrc_siaf = pysiaf.Siaf('NIRCAM') 


In [74]:
def initplot(nrows=1, ncols=1, xfigsize4subplot=5, yfigsize4subplot=5, **kwargs):
    sp=[]
    xfigsize=xfigsize4subplot*ncols
    yfigsize=yfigsize4subplot*nrows
    plt.figure(figsize=(xfigsize,yfigsize))
    counter=1
    for row in range(nrows):
        for col in range(ncols):
            sp.append(plt.subplot(nrows, ncols, counter,**kwargs))
            counter+=1

    for i in range(len(sp)):
        plt.setp(sp[i].get_xticklabels(),'fontsize',12)
        plt.setp(sp[i].get_yticklabels(),'fontsize',12)
        sp[i].set_xlabel(sp[i].get_xlabel(),fontsize=14)
        sp[i].set_ylabel(sp[i].get_ylabel(),fontsize=14)
        sp[i].set_title(sp[i].get_title(),fontsize=14)

    return(sp)


In [77]:
mean_results = pdastrostatsclass()
progIDs0 = None
apertures0 = None
filts0=None
v2v3params=['V3IdlYAngle','V2Ref','V3Ref']
format_params = '{:.8f}'
verbose=1

progIDs0=['04447','01476','all']
apertures0=['nrca1_full']
#filts0=['f150w']

dbin={}
dbin['V3IdlYAngle'] = 0.03
dbin['V2Ref'] = 0.15
dbin['V3Ref'] = 0.15
binsize={}
binsize['V3IdlYAngle']=0.002
binsize['V2Ref']=0.01
binsize['V3Ref']=0.01
colors={}
colors['04447']='blue'
colors['01476']='red'
colors['all']='cyan'
histtypes={}
histtypes['04447']='step'
histtypes['01476']='step'
histtypes['all']='bar'
major_ticksize={}
major_ticksize['V3IdlYAngle']=0.01
major_ticksize['V2Ref']=0.1
major_ticksize['V3Ref']=0.1
minor_ticksize={}
minor_ticksize['V3IdlYAngle']=0.001
minor_ticksize['V2Ref']=0.01
minor_ticksize['V3Ref']=0.01


if apertures0 is None:
    apertures = unique(v2v3.t['aperture'])
else:
    apertures = apertures0
apertures.sort()

for aperture in apertures:
    if verbose: print(f'### apertures {aperture}')
    # get entries for aperture
    ixs_aper = v2v3.ix_equal('aperture',aperture.lower())
    if len(ixs_aper)==0:
        print(f'Warning: skipping aperture {aperture} since there are no entries!')
        continue

    if filts0 is None:
        filts = unique(v2v3.t.loc[ixs_aper,'filter'])
    else:
        filts = filts0
    filts.sort()

    sp=initplot(len(filts),len(v2v3params))
    
    vals_siaf = {}
    vals_siaf['V3IdlYAngle']=nrc_siaf[aperture.upper()].V3IdlYAngle
    vals_siaf['V2Ref']=nrc_siaf[aperture.upper()].V2Ref
    vals_siaf['V3Ref']=nrc_siaf[aperture.upper()].V3Ref
    
    xlims = {}
    xlims['V3IdlYAngle']= (nrc_siaf[aperture.upper()].V3IdlYAngle-dbin['V3IdlYAngle'],
                           nrc_siaf[aperture.upper()].V3IdlYAngle+dbin['V3IdlYAngle'])
    xlims['V2Ref']= (nrc_siaf[aperture.upper()].V2Ref-dbin['V2Ref'],
                     nrc_siaf[aperture.upper()].V2Ref+dbin['V2Ref'])
    xlims['V3Ref']= (nrc_siaf[aperture.upper()].V3Ref-dbin['V3Ref'],
                     nrc_siaf[aperture.upper()].V3Ref+dbin['V3Ref'])

    bins = {}
    bins['V3IdlYAngle']= np.arange(xlims['V3IdlYAngle'][0]-binsize['V3IdlYAngle'],
                                   xlims['V3IdlYAngle'][1]+binsize['V3IdlYAngle'],binsize['V3IdlYAngle'])
    bins['V2Ref']= np.arange(xlims['V2Ref'][0]-binsize['V3IdlYAngle'],
                             xlims['V2Ref'][1]+binsize['V3IdlYAngle'],binsize['V2Ref'])
    bins['V3Ref']= np.arange(xlims['V3Ref'][0]-binsize['V3IdlYAngle'],
                             xlims['V3Ref'][1]+binsize['V3IdlYAngle'],binsize['V3Ref'])

    for filtercounter, filt in enumerate(filts):
    #for filt in filts:
        ixs_filt = v2v3.ix_equal('filter',filt.lower(),indices=ixs_aper)
        if len(ixs_filt)==0:
            print(f'Warning: skipping filter {filt} for aperture {aperture} since there are no entries!')
            continue

        if progIDs0 is None:
            progIDs = unique(v2v3.t.loc[ixs_filt,'progID'])
            progIDs.append('all')
        else:
            progIDs = progIDs0
        progIDs.sort()
                
        ixs_progID_all = []
        for progID in progIDs:
            if progID != 'all':
                ixs_progID = v2v3.ix_equal('progID',progID,indices=ixs_filt)
                if len(ixs_progID)==0:
                    print(f'Warning: skipping progID {progID} for filter {filt} and aperture {aperture} since there are no entries!')
                    continue
                ixs_progID_all.extend(ixs_progID)
            else:
                # use all indices for this filter for the final fit!!
                ixs_progID = ixs_progID_all

            # for each aperture, filter, and progID there is one row in the results
            ix_results = mean_results.newrow({'aperture':aperture,'filter':filt,'progID':progID})


            if verbose>2:
                v2v3.write(indices=ixs_progID,columns=['aperture','filter','V3IdlYAngle','V2Ref','V3Ref'])

            for paramcounter,v2v3paramname in enumerate(v2v3params):
                v2v3.calcaverage_sigmacutloop(v2v3paramname,indices=ixs_progID,percentile_cut_firstiteration=60)

                mean_results.param2columnmapping = mean_results.intializecols4statparams(
                    setcol2None=False,
                    params=['mean','mean_err','stdev','Ngood','Nclip'], # define the statistic parameters that you want to keep
                    prefix=f'{v2v3paramname}_', # define the prefix for the output statistics columns, e.g. for mean it will be f'{prefix}mean'
                    format4outvals=format_params) # format of statistics columns

                mean_results.statresults2table(v2v3.statparams,
                                               mean_results.param2columnmapping,
                                               destindex=ix_results)
                
                # plotting histograms!
                sp[filtercounter*len(v2v3params)+paramcounter].hist(v2v3.t.loc[ixs_progID,v2v3paramname], 
                                                                    bins=bins[v2v3paramname], 
                                                                    color=colors[progID],
                                                                    histtype=histtypes[progID])
                                                                    #histtype='barstacked')
                sp[filtercounter*len(v2v3params)+paramcounter].set_title(f'{aperture} {filt} {v2v3paramname}')
                sp[filtercounter*len(v2v3params)+paramcounter].set_xlim(xlims[v2v3paramname])
                sp[filtercounter*len(v2v3params)+paramcounter].yaxis.set_major_locator(MultipleLocator(1))
                sp[filtercounter*len(v2v3params)+paramcounter].xaxis.set_major_locator(MultipleLocator(major_ticksize[v2v3paramname]))
                sp[filtercounter*len(v2v3params)+paramcounter].xaxis.set_minor_locator(MultipleLocator(minor_ticksize[v2v3paramname]))
                sp[filtercounter*len(v2v3params)+paramcounter].set_xlabel(v2v3paramname)
                
                if progID=='all':
                    sp[filtercounter*len(v2v3params)+paramcounter].axvline(mean_results.t.loc[ix_results,f'{v2v3paramname}_mean'],  color='black',linestyle='-', linewidth=3.0)
                    sp[filtercounter*len(v2v3params)+paramcounter].axvline(vals_siaf[v2v3paramname],  color='red',linestyle='-', linewidth=3.0)

                
    plt.tight_layout()
    #plt.show()
    
    outputfilename = re.sub('\.txt',f'.{aperture}.png',v2v3.filename)
    if outputfilename == v2v3.filename:
        raise RuntimeError(f'could not determine outputfilename from {v2v3.filename}')
    print(f'Saving histograms for aperture {aperture} into {outputfilename}')
    if os.path.isfile(outputfilename):
        rmfile(outputfilename)

    plt.savefig(outputfilename)
    plt.close()
    
mean_results.write(columns=['aperture','filter','progID','V3Ref_mean','V3Ref_mean_err','V3Ref_stdev','V3Ref_Ngood','V3Ref_Nclip'])
outputfilename = re.sub('\.txt','.mean.txt',v2v3.filename)
if outputfilename == v2v3.filename:
    raise RuntimeError(f'could not determine outputfilename from {v2v3.filename}')
print(f'Saving averages into {outputfilename}')
mean_results.write(outputfilename)


### apertures nrca1_full
Saving histograms for aperture nrca1_full into nircam_v2v3_v3.nrca1_full.png
  aperture filter progID    V3Ref_mean V3Ref_mean_err V3Ref_stdev V3Ref_Ngood V3Ref_Nclip
nrca1_full  f070w  04447 -527.50535860     0.00164385  0.00328771           5           3
nrca1_full  f070w    all -527.50535860     0.00164385  0.00328771           5           3
nrca1_full  f150w  01476 -527.50121043     0.00231212  0.00566351           7           1
nrca1_full  f150w  04447 -527.50288614     0.00237263  0.00581174           7           1
nrca1_full  f150w    all -527.50204829     0.00154793  0.00558113          14           2
nrca1_full  f200w  01476 -527.63852062     0.00269891  0.00714064           8           0
nrca1_full  f200w  04447 -527.63114363     0.00299702  0.00792936           8           0
nrca1_full  f200w    all -527.63483213     0.00212365  0.00822486          16           0
nrca1_full  f210m  01476 -527.50366825     0.00274639  0.00726626           8           

0